In [ ]:
%pip install -Uqqq datasets openinference-instrumentation-openai openai-responses openai tiktoken langchain langchain-openai llama-index llama-index-llms-openai faker mdgen

In [ ]:
from contextlib import ExitStack
from random import randint, random, shuffle

import openai
from datasets import load_dataset
from faker import Faker
from mdgen import MarkdownPostProvider
from openai_responses import OpenAIMock
from openinference.instrumentation import using_session, using_user
from openinference.instrumentation.openai import OpenAIInstrumentor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import SpanLimits, TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk.trace.export.in_memory_span_exporter import InMemorySpanExporter
from tiktoken import encoding_for_model

fake = Faker()
fake.add_provider(MarkdownPostProvider)

# Download Data

In [ ]:
df = load_dataset("GitBag/ultrainteract_multiturn_1_iter_processed_harvard")["train"].to_pandas()
convo = df.loc[df.chosen.apply(len) == 10, "chosen"]

# Tracer Provider

In [ ]:
tracer_provider = TracerProvider(span_limits=SpanLimits(max_attributes=1_000_000))
in_memory_span_exporter = InMemorySpanExporter()
tracer_provider.add_span_processor(SimpleSpanProcessor(in_memory_span_exporter))
endpoint = "http://127.0.0.1:4317"
otlp_span_exporter = OTLPSpanExporter(endpoint=endpoint)

# Helpers

In [ ]:
def gen_session_id():
    p = random()
    if p < 0.1:
        return ":" * randint(1, 5)
    if p < 0.9:
        return Faker(["ja_JP", "vi_VN", "ko_KR", "zh_CN"]).address()
    return int(abs(random()) * 1_000_000_000)


def gen_user_id():
    p = random()
    if p < 0.1:
        return ":" * randint(1, 5)
    if p < 0.9:
        return Faker(["ja_JP", "vi_VN", "ko_KR", "zh_CN"]).name()
    return int(abs(random()) * 1_000_000_000)


def export_spans():
    """Export spans in random order for receiver testing"""
    spans = list(in_memory_span_exporter.get_finished_spans())
    shuffle(spans)
    for span in spans:
        otlp_span_exporter.export([span])
    in_memory_span_exporter.clear()
    session_count = len({id_ for span in spans if (id_ := span.attributes.get("session.id"))})
    trace_count = len({span.context.trace_id for span in spans})
    print(f"Exported {session_count} sessions, {trace_count} traces, {len(spans)} spans")

# Text Only

## OpenAI

In [ ]:
session_count = randint(10, 20)


def simulate_openai(messages):
    user_id = gen_user_id() if random() < 0.9 else " "
    session_id = gen_session_id()
    client = openai.Client(api_key="sk-")
    model = "gpt-4o-mini"
    encoding = encoding_for_model(model)
    counts = [len(encoding.encode(m["content"])) for m in messages]
    multiplier = randint(1, 10)
    messages = list(messages) * multiplier
    counts = counts * multiplier
    openai_mock = OpenAIMock()
    tracer = tracer_provider.get_tracer(__name__)
    with openai_mock.router:
        for i in range(1, len(messages), 2):
            openai_mock.chat.completions.create.response = dict(
                choices=[dict(index=0, finish_reason="stop", message=messages[i])],
                usage=dict(
                    prompt_tokens=sum(counts[:i]),
                    completion_tokens=counts[i],
                    total_tokens=sum(counts[: i + 1]),
                ),
            )
            with ExitStack() as stack:
                attributes = {
                    "input.value": messages[i - 1]["content"],
                    "output.value": messages[i]["content"],
                }
                if random() < 0.5:
                    attributes["session.id"] = session_id
                    attributes["user.id"] = user_id
                else:
                    stack.enter_context(using_session(session_id))
                    stack.enter_context(using_user(user_id))
                stack.enter_context(tracer.start_as_current_span("root", attributes=attributes))
                client.chat.completions.create(model=model, messages=messages[:i])


OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
convo.sample(session_count).apply(simulate_openai)
OpenAIInstrumentor().uninstrument()
export_spans()